In [2]:
from urllib.request import Request, urlopen, urlretrieve
from urllib.parse import urljoin
import requests
import re
import os
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

rootdir = "C:/Users/minif/OneDrive/Desktop/"
data = pd.read_excel(rootdir + "Pharma_historical.xlsx")

In [2]:
def pull_html(ticker, start_date, end_date):
    try:
        r = requests.get("https://finance.yahoo.com/quote/" + ticker + "/history?period1=" + str(start_date) + "&period2=" + str(end_date) + "&interval=1d&filter=history&frequency=1d")
        
    except Exception :
        print("An internet error has occured.")
        return

    if r.status_code == 200:
        r = Request("https://finance.yahoo.com/quote/" + ticker + "/history?period1=" + str(start_date) + "&period2=" + str(end_date) + "&interval=1d&filter=history&frequency=1d")
        print("https://finance.yahoo.com/quote/" + ticker + "/history?period1=" + str(start_date) + "&period2=" + str(end_date) + "&interval=1d&filter=history&frequency=1d")
        u = urlopen(r)
        html = u.read().decode('utf-8')
        return html
    else:
        print("An HTTP Error has Occured with pull_html(" + ticker + ") Error Code: "+str(r.status_code))

def convert_date_short(date):
    return datetime.utcfromtimestamp(date).strftime('%b %d, %Y')

def clean_html(html):
    output = ""
    old_index = 0
    done = False
    
    if "data-reactid=" in html:
        while done == False:
            index = html.find("data-reactid=",old_index)
            if index == -1:
                done = True
                break
            output += html[old_index:index-1]
            old_index = html.find(">",index)
    
    return output

In [3]:
def find_open_close(html, start_date, end_date):
    date_start_f = convert_date_short(start_date)
    date_end_f = convert_date_short(end_date)
    
    # find the indexes
    index_end_date = html.find("<span>" + date_end_f + "</span>")
    index_start_date = html.find("<span>" + date_start_f + "</span>")
    
    #print(date_end_f,index_end_date)
    #print(date_start_f,index_start_date)
    
    #find open, high, low, and close prices for the start date
    index_open_start_1 = html.find("<span>",index_start_date + 25)
    index_open_start_2 = html.find("</span>",index_open_start_1)
    
    #print(index_open_start_1)
    #print(index_open_start_2)
    
    open_start_price = html[index_open_start_1 + 6: index_open_start_2]
    
    #rinse and repeat for the rest
    index_high_start_1 = html.find("<span>",index_open_start_2)
    index_high_start_2 = html.find("</span>",index_high_start_1)
    
    high_start_price = html[index_high_start_1 + 6:index_high_start_2]
    
    index_low_start_1 = html.find("<span>", index_high_start_2)
    index_low_start_2 = html.find("</span>", index_low_start_1)
    
    low_start_price = html[index_low_start_1 + 6:index_low_start_2]
    
    index_close_start_1 = html.find("<span>",index_low_start_2)
    index_close_start_2 = html.find("</span>",index_close_start_1)
    
    close_start_price = html[index_close_start_1 + 6:index_close_start_2]
    
    #find open, high, low, and close prices for the end date
    index_open_end_1 = html.find("<span>",index_end_date + 25)
    index_open_end_2 = html.find("</span>",index_open_end_1)
    
    open_end_price = html[index_open_end_1 + 6:index_open_end_2]
    #rinse and repeat for the rest
    index_high_end_1 = html.find("<span>",index_open_end_2)
    index_high_end_2 = html.find("</span>",index_high_end_1)
    
    high_end_price = html[index_high_end_1 + 6:index_high_end_2]
    
    index_low_end_1 = html.find("<span>",index_high_end_2)
    index_low_end_2 = html.find("</span>",index_low_end_1)
    
    low_end_price = html[index_low_end_1 + 6:index_low_end_2]
    
    index_close_end_1 = html.find("<span>",index_low_end_2)
    index_close_end_2 = html.find("</span>",index_close_end_1)
    
    close_end_price = html[index_close_end_1 + 6:index_close_end_2]
    
    #open start, high start, low start, close start, open end, high end, low end, close end
    return [open_start_price, high_start_price, low_start_price, close_start_price, open_end_price, high_end_price, low_end_price, close_end_price]
    

In [4]:
tickers = data["Tickers"].values.tolist()
start_dates_unix = data["Unix Timestamp"].values.tolist()
end_dates_unix = data["Unix Timestamp After"].values.tolist()

ticker_test = tickers[0:len(tickers)]

start_dates_formatted = []
end_dates_formatted = []

open_start_prices = []
high_start_prices = []
low_start_prices = []
close_start_prices = []
open_end_prices = []
high_end_prices = []
low_end_prices = []
close_end_prices = []

In [5]:
def correct_date(date_start,date_end):
    date_start = convert_date_short(date_start)
    date_end = convert_date_short(date_end)
    
    #saturday
    date_start = datetime.strptime(date_start, '%b %d, %Y')
    date_end = datetime.strptime(date_end, '%b %d, %Y')
    
    if date_start.weekday() == 5:
        date_start += timedelta(days=2)
        date_end += timedelta(days=2)
    #sunday
    if date_start.weekday() == 6:
        date_start += timedelta(days=1)
        date_end += timedelta(days=1)
    
    #end is saturday
    if date_end.weekday() == 5:
        date_end += timedelta(days=2)
    #end is sunday
    if date_end.weekday() == 6:
        date_end += timedelta(days=2)
    if date_end.weekday() == 0:
        date_end += timedelta(days=1)
    
    return [date_start.timestamp(),date_end.timestamp()]

In [6]:
for t in range(len(ticker_test)):
    correct_dates_unix = correct_date(start_dates_unix[t],end_dates_unix[t])
    correct_dates_unix[0] = int(correct_dates_unix[0])
    correct_dates_unix[1] = int(correct_dates_unix[1])
    
    start_dates_formatted.append(convert_date_short(correct_dates_unix[0]))
    end_dates_formatted.append(convert_date_short(correct_dates_unix[1]))
    
    print(t)
    
    html_output = pull_html(ticker_test[t],correct_dates_unix[0],correct_dates_unix[1])
    #print(html_output)
    #print(clean_html(html_output))
    prices = find_open_close(clean_html(html_output),correct_dates_unix[0],correct_dates_unix[1])
    open_start_prices.append(prices[0])
    high_start_prices.append(prices[1])
    low_start_prices.append(prices[2])
    close_start_prices.append(prices[3])
    open_end_prices.append(prices[4])
    high_end_prices.append(prices[5])
    low_end_prices.append(prices[6])
    close_end_prices.append(prices[7])

0
https://finance.yahoo.com/quote/SPPI/history?period1=1246852800&period2=1247112000&interval=1d&filter=history&frequency=1d
1
https://finance.yahoo.com/quote/ZGNX/history?period1=1247716800&period2=1248148800&interval=1d&filter=history&frequency=1d
2
https://finance.yahoo.com/quote/ARDM/history?period1=1247716800&period2=1248148800&interval=1d&filter=history&frequency=1d
3
https://finance.yahoo.com/quote/SPPI/history?period1=1252036800&period2=1252382400&interval=1d&filter=history&frequency=1d
4
https://finance.yahoo.com/quote/SPPI/history?period1=1255060800&period2=1255406400&interval=1d&filter=history&frequency=1d
5
https://finance.yahoo.com/quote/PRTK/history?period1=1257138000&period2=1257397200&interval=1d&filter=history&frequency=1d
6
https://finance.yahoo.com/quote/GTXI/history?period1=1257138000&period2=1257397200&interval=1d&filter=history&frequency=1d
7
https://finance.yahoo.com/quote/HEB/history?period1=1259643600&period2=1259902800&interval=1d&filter=history&frequency=1d
8

66
https://finance.yahoo.com/quote/BMRN/history?period1=1392354000&period2=1392699600&interval=1d&filter=history&frequency=1d
67
https://finance.yahoo.com/quote/ENDP/history?period1=1394082000&period2=1394510400&interval=1d&filter=history&frequency=1d
68
https://finance.yahoo.com/quote/ARLZ/history?period1=1398398400&period2=1398744000&interval=1d&filter=history&frequency=1d
69
https://finance.yahoo.com/quote/MDCO/history?period1=1398830400&period2=1399348800&interval=1d&filter=history&frequency=1d
70
https://finance.yahoo.com/quote/OMER/history?period1=1401681600&period2=1401940800&interval=1d&filter=history&frequency=1d
71
https://finance.yahoo.com/quote/BDSI/history?period1=1402027200&period2=1402372800&interval=1d&filter=history&frequency=1d
72
https://finance.yahoo.com/quote/NAVB/history?period1=1402632000&period2=1402977600&interval=1d&filter=history&frequency=1d
73
https://finance.yahoo.com/quote/MNKD/history?period1=1403841600&period2=1404187200&interval=1d&filter=history&frequ

131
https://finance.yahoo.com/quote/NEOS/history?period1=1453870800&period2=1454389200&interval=1d&filter=history&frequency=1d
132
https://finance.yahoo.com/quote/VRTX/history?period1=1454648400&period2=1454994000&interval=1d&filter=history&frequency=1d
133
https://finance.yahoo.com/quote/NBIX/history?period1=1455080400&period2=1455598800&interval=1d&filter=history&frequency=1d
134
https://finance.yahoo.com/quote/ABBV/history?period1=1455080400&period2=1455598800&interval=1d&filter=history&frequency=1d
135
https://finance.yahoo.com/quote/RDHL/history?period1=1457413200&period2=1457672400&interval=1d&filter=history&frequency=1d
136
https://finance.yahoo.com/quote/GWPH/history?period1=1457928000&period2=1458187200&interval=1d&filter=history&frequency=1d
137
https://finance.yahoo.com/quote/SPPI/history?period1=1458014400&period2=1458273600&interval=1d&filter=history&frequency=1d
138
https://finance.yahoo.com/quote/EGRX/history?period1=1458273600&period2=1458619200&interval=1d&filter=histo

196
https://finance.yahoo.com/quote/ABUS/history?period1=1475121600&period2=1475553600&interval=1d&filter=history&frequency=1d
197
https://finance.yahoo.com/quote/MLNT/history?period1=1475208000&period2=1475553600&interval=1d&filter=history&frequency=1d
198
https://finance.yahoo.com/quote/TENX/history?period1=1475640000&period2=1476158400&interval=1d&filter=history&frequency=1d
199
https://finance.yahoo.com/quote/ALNY/history?period1=1475640000&period2=1476158400&interval=1d&filter=history&frequency=1d
200
https://finance.yahoo.com/quote/ALKS/history?period1=1476936000&period2=1477368000&interval=1d&filter=history&frequency=1d
201
https://finance.yahoo.com/quote/PLXP/history?period1=1477368000&period2=1477627200&interval=1d&filter=history&frequency=1d
202
https://finance.yahoo.com/quote/RIGL/history?period1=1477886400&period2=1478145600&interval=1d&filter=history&frequency=1d
203
https://finance.yahoo.com/quote/RIGL/history?period1=1477886400&period2=1478145600&interval=1d&filter=histo

261
https://finance.yahoo.com/quote/NBIX/history?period1=1484715600&period2=1485234000&interval=1d&filter=history&frequency=1d
262
https://finance.yahoo.com/quote/AGN/history?period1=1484715600&period2=1485234000&interval=1d&filter=history&frequency=1d
263
https://finance.yahoo.com/quote/AGN/history?period1=1484802000&period2=1485234000&interval=1d&filter=history&frequency=1d
264
https://finance.yahoo.com/quote/ABBV/history?period1=1484802000&period2=1485234000&interval=1d&filter=history&frequency=1d
265
https://finance.yahoo.com/quote/SGYP/history?period1=1484802000&period2=1485234000&interval=1d&filter=history&frequency=1d
266
https://finance.yahoo.com/quote/MRNS/history?period1=1485147600&period2=1485406800&interval=1d&filter=history&frequency=1d
267
https://finance.yahoo.com/quote/VRTX/history?period1=1485320400&period2=1485838800&interval=1d&filter=history&frequency=1d
268
https://finance.yahoo.com/quote/AGN/history?period1=1485406800&period2=1485838800&interval=1d&filter=history&

...

1171
https://finance.yahoo.com/quote/ABIO/history?period1=1527480000&period2=1527739200&interval=1d&filter=history&frequency=1d
1172
https://finance.yahoo.com/quote/RHHB/history?period1=1527566400&period2=1527825600&interval=1d&filter=history&frequency=1d
1173
https://finance.yahoo.com/quote/IMMP/history?period1=1527652800&period2=1528171200&interval=1d&filter=history&frequency=1d
1174
https://finance.yahoo.com/quote/VRTX/history?period1=1527652800&period2=1528171200&interval=1d&filter=history&frequency=1d
1175
https://finance.yahoo.com/quote/CRSP/history?period1=1527652800&period2=1528171200&interval=1d&filter=history&frequency=1d
1176
https://finance.yahoo.com/quote/GLPG/history?period1=1527652800&period2=1528171200&interval=1d&filter=history&frequency=1d
1177
https://finance.yahoo.com/quote/GILD/history?period1=1527652800&period2=1528171200&interval=1d&filter=history&frequency=1d
1178
https://finance.yahoo.com/quote/AZN/history?period1=1527652800&period2=1528171200&interval=1d&filte

1236
https://finance.yahoo.com/quote/MEIP/history?period1=1528084800&period2=1528344000&interval=1d&filter=history&frequency=1d
1237
https://finance.yahoo.com/quote/BMY/history?period1=1528084800&period2=1528344000&interval=1d&filter=history&frequency=1d
1238
https://finance.yahoo.com/quote/PLX/history?period1=1528171200&period2=1528430400&interval=1d&filter=history&frequency=1d
1239
https://finance.yahoo.com/quote/LLY/history?period1=1528171200&period2=1528430400&interval=1d&filter=history&frequency=1d
1240
https://finance.yahoo.com/quote/MRK/history?period1=1528171200&period2=1528430400&interval=1d&filter=history&frequency=1d
1241
https://finance.yahoo.com/quote/LLY/history?period1=1528171200&period2=1528430400&interval=1d&filter=history&frequency=1d
1242
https://finance.yahoo.com/quote/ABBV/history?period1=1528171200&period2=1528430400&interval=1d&filter=history&frequency=1d
1243
https://finance.yahoo.com/quote/PTI/history?period1=1528257600&period2=1528776000&interval=1d&filter=his

1301
https://finance.yahoo.com/quote/PTIE/history?period1=1529985600&period2=1530244800&interval=1d&filter=history&frequency=1d
1302
https://finance.yahoo.com/quote/SMMT/history?period1=1530072000&period2=1530590400&interval=1d&filter=history&frequency=1d
1303
https://finance.yahoo.com/quote/GBT/history?period1=1530072000&period2=1530590400&interval=1d&filter=history&frequency=1d
1304
https://finance.yahoo.com/quote/ARRY/history?period1=1530072000&period2=1530590400&interval=1d&filter=history&frequency=1d
1305
https://finance.yahoo.com/quote/AQXP/history?period1=1530072000&period2=1530590400&interval=1d&filter=history&frequency=1d
1306
https://finance.yahoo.com/quote/AZN/history?period1=1530072000&period2=1530590400&interval=1d&filter=history&frequency=1d
1307
https://finance.yahoo.com/quote/MRK/history?period1=1530072000&period2=1530590400&interval=1d&filter=history&frequency=1d
1308
https://finance.yahoo.com/quote/CARA/history?period1=1530072000&period2=1530590400&interval=1d&filter=

1366
https://finance.yahoo.com/quote/RDHL/history?period1=1532923200&period2=1533182400&interval=1d&filter=history&frequency=1d
1367
https://finance.yahoo.com/quote/PGNX/history?period1=1532923200&period2=1533182400&interval=1d&filter=history&frequency=1d
1368
https://finance.yahoo.com/quote/TBPH/history?period1=1533096000&period2=1533614400&interval=1d&filter=history&frequency=1d
1369
https://finance.yahoo.com/quote/SBPH/history?period1=1533182400&period2=1533614400&interval=1d&filter=history&frequency=1d
1370
https://finance.yahoo.com/quote/EPZM/history?period1=1533182400&period2=1533614400&interval=1d&filter=history&frequency=1d
1371
https://finance.yahoo.com/quote/OVID/history?period1=1533528000&period2=1533787200&interval=1d&filter=history&frequency=1d
1372
https://finance.yahoo.com/quote/PTGX/history?period1=1533528000&period2=1533787200&interval=1d&filter=history&frequency=1d
1373
https://finance.yahoo.com/quote/DRRX/history?period1=1533528000&period2=1533787200&interval=1d&filt

1431
https://finance.yahoo.com/quote/ATOS/history?period1=1536811200&period2=1537243200&interval=1d&filter=history&frequency=1d
1432
https://finance.yahoo.com/quote/TEVA/history?period1=1536897600&period2=1537243200&interval=1d&filter=history&frequency=1d
1433
https://finance.yahoo.com/quote/ARGX/history?period1=1537156800&period2=1537416000&interval=1d&filter=history&frequency=1d
1434
https://finance.yahoo.com/quote/ZEAL/history?period1=1537243200&period2=1537502400&interval=1d&filter=history&frequency=1d
1435
https://finance.yahoo.com/quote/VKTX/history?period1=1537243200&period2=1537502400&interval=1d&filter=history&frequency=1d
1436
https://finance.yahoo.com/quote/GALT/history?period1=1537416000&period2=1537848000&interval=1d&filter=history&frequency=1d
1437
https://finance.yahoo.com/quote/GTXI/history?period1=1537502400&period2=1537848000&interval=1d&filter=history&frequency=1d
1438
https://finance.yahoo.com/quote/NITE/history?period1=1537761600&period2=1538020800&interval=1d&filt

In [7]:
def write_data_excel(data):
    df_output = pd.DataFrame({'Tickers': data[0], 'Market Start Date': data[1], 'Market End Date': data[2], 'Open Start': data[3], 'High Start': data[4], 'Low Start': data[5], 'Close Start': data[6], 'Open End': data[7], 'High End': data[8], 'Low End': data[9], 'Close End': data[10]})
    
    writer = pd.ExcelWriter('C:/Users/minif/OneDrive/Desktop/stock_data_pharma_yahoo' + str(len(ticker_test)) + '.xlsx')
    df_output.to_excel(writer,'Sheet1')
    writer.save()
    
    return df_output

In [8]:
final_data = [ticker_test,start_dates_formatted,end_dates_formatted,open_start_prices,high_start_prices,low_start_prices,close_start_prices,open_end_prices,high_end_prices,low_end_prices,close_end_prices]
write_data_excel(final_data)

,Tickers,Market Start Date,Market End Date,Open Start,High Start,Low Start,Close Start,Open End,High End,Low End,Close End
0,SPPI,"Jul 06, 2009","Jul 09, 2009",5.78,5.96,5.30,5.40,5.30,5.43,5.13,5.23
1,ZGNX,"Jul 16, 2009","Jul 21, 2009",Now you can search stock related news and priv...,No matching results for &#x27;&#x27;,Tip: Try a valid symbol or a specific company ...,Give feedback on the new search experience,Now you can search stock related news and priv...,No matching results for &#x27;&#x27;,Tip: Try a valid symbol or a specific company ...,Give feedback on the new search experience
2,ARDM,"Jul 16, 2009","Jul 21, 2009",8.40,12.80,8.40,11.20,8.00,8.80,8.00,8.40
3,SPPI,"Sep 04, 2009","Sep 08, 2009",8.94,10.00,7.90,8.33,8.19,8.22,6.93,7.19
4,SPPI,"Oct 09, 2009","Oct 13, 2009",4.98,5.44,4.82,5.09,4.86,4.88,4.51,4.56
5,PRTK,"Nov 02, 2009","Nov 05, 2009",63.00,66.00,60.96,64.80,55.32,65.40,53.40,64.80
6,GTXI,"Nov 02, 2009","Nov 05, 2009",58.00,59.00,44.90,45.60,37.20,39.90,35.80,38.10
7,HEB,"Dec 01, 2009","Dec 04, 2009",14.6400,14.8800,14.0400,14.4000,8.0400,8.2800,7.3200,7.4400
8,FCSC,"Dec 18, 2009","Dec 22, 2009",457.50,457.50,450.00,450.00,393.75,427.50,375.00,427.50
9,ACOR,"Jan 22, 2010","Jan 26, 2010",25.71,29.36,25.10,28.12,27.79,27.96,27.05,27.48
